# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA (Low-Rank Adaptation)
    * Greatly reduces the parameters that need to be trained during fine-tuning.
* Model: GPT-2
    * Familiar with GPT-2.
* Evaluation approach: Hugging Face Trainer's evaluate method
    * Simplifies the training and evaluation process.
* Fine-tuning dataset: Hugging Face Twitter Financial News Sentiment
    * Just wrapped up the AI for Trading nanodegree so I decided to use this financial dataset.

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Load Dataset

In [1]:
# Load the twitter-financial-news-sentiment dataset
# See: https://huggingface.co/datasets/zeroshot/twitter-financial-news-sentiment
from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "validation"]
ds = {split: ds for split, ds in zip(splits, load_dataset("zeroshot/twitter-financial-news-sentiment", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=23).select(range(2000))

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 2000
 }),
 'validation': Dataset({
     features: ['text', 'label'],
     num_rows: 2000
 })}

In [2]:
ds['train'][:5]

{'text': ['Ray Dalio isn’t disputing the investment that WSJ reported. But... he is definitely taking issue with the WSJ headl… https://t.co/tUmnpPET7o',
  'UKDMO UK Regulatory Announcement: Issue of Debt https://t.co/NFYxKNlKdM https://t.co/KpBYsXN3wZ',
  'China Sends New Officials to Take Charge of Hubei Outbreak',
  'Digital Turbine -5% after Q3 revenue miss',
  'Trade deal removes major hurdle for rally in Apple and tech'],
 'label': [2, 2, 2, 0, 1]}

### Load Tokenizer

In [3]:
from transformers import AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Let's use a lambda function to tokenize all the examples
tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(
        lambda x: tokenizer(x["text"], padding="max_length", truncation=True), batched=True
    )

# Inspect the available columns in the dataset
tokenized_ds["train"]


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2000
})

In [4]:
tokenized_ds

{'train': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 2000
 }),
 'validation': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 2000
 })}

### Load Model

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label={0: "Bearish", 1: "Bullish", 2: "Neutral"},
    label2id={"Bearish": 0, "Bullish": 1, "Neutral": 2},
)
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=3, bias=False)
)


### Train the Model

In [7]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/results",
        learning_rate=1e-4,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

#trainer.train()

### Evaluate the Model

In [8]:
original_results = trainer.evaluate()
print(original_results)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 2.8927557468414307, 'eval_accuracy': 0.4315, 'eval_runtime': 178.8924, 'eval_samples_per_second': 11.18, 'eval_steps_per_second': 2.795}


### View the Results

In [9]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd

items_for_manual_review = tokenized_ds["validation"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

results = trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "text": [item["text"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,text,predictions,labels
0,BioLineRx reports data on triple combination arm of COMBAT/KEYNOTE-202 study; shares up 21% premarket,2,1
1,NYC pension leader targets three utilities over emissions plans,2,2
2,Oil Prices' Hard Landing Initializing Descent. Continue reading: https://t.co/4zqKuKP70N #markets #business #economy,0,0
3,'Bloomberg The Open' Full Show (12/13/2019),2,2
4,https://t.co/LKXMOP3zNK Inc. was on course to exceed $1 trillion in market value again after results beat expectati... https://t.co/4tiFA4prdg,0,1
5,$IART - Integra LifeSciences sees drop in Q1 revenue amid COVID-19 https://t.co/E7Eo6ADSvu,0,0
6,Fed's Rosengren backs 'patient' approach to any interest-rate moves,2,2
7,Alliance Data taps new CEO from Citi,2,2


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [10]:
from peft import LoraConfig, TaskType
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS
)

In [11]:
#from transformersimport AutoModelForCausalLM
#model = AutoModelForCausalLM.from_pretrained("gpt2")

In [12]:
from peft import get_peft_model
lora_model = get_peft_model(model, peft_config)

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [13]:
lora_model.print_trainable_parameters()

trainable params: 299,520 || all params: 124,739,328 || trainable%: 0.2401167336736013


In [14]:
tokenized_ds['train']

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2000
})

In [15]:
for split in splits:
    #tokenized_ds[split] = tokenized_ds[split].rename_column("labels", "label")
    tokenized_ds[split].set_format(type='torch', columns=['label', 'input_ids', 'attention_mask'])

In [16]:
training_args = TrainingArguments(
    output_dir="./data/results",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.306500,0.857798,0.656500
2,0.860600,0.807315,0.679500


Checkpoint destination directory ./data/results/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1000, training_loss=1.0835752868652344, metrics={'train_runtime': 1211.3653, 'train_samples_per_second': 3.302, 'train_steps_per_second': 0.826, 'total_flos': 2097697259520000.0, 'train_loss': 1.0835752868652344, 'epoch': 2.0})

In [17]:
lora_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [22]:
from peft import PeftModelForSequenceClassification
lora_model = PeftModelForSequenceClassification.from_pretrained(model, "gpt-lora")

In [29]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [30]:
training_args = TrainingArguments(
    output_dir="./data/results",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

In [31]:
fine_tuned_results = trainer.evaluate()
print(fine_tuned_results)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.8073154091835022, 'eval_accuracy': 0.6795, 'eval_runtime': 183.0701, 'eval_samples_per_second': 10.925, 'eval_steps_per_second': 2.731}


In [32]:
print("Original Model Performance:", original_results)
print("Fine-Tuned Model Performance:", fine_tuned_results)

Original Model Performance: {'eval_loss': 2.8927557468414307, 'eval_accuracy': 0.4315, 'eval_runtime': 178.8924, 'eval_samples_per_second': 11.18, 'eval_steps_per_second': 2.795}
Fine-Tuned Model Performance: {'eval_loss': 0.8073154091835022, 'eval_accuracy': 0.6795, 'eval_runtime': 183.0701, 'eval_samples_per_second': 10.925, 'eval_steps_per_second': 2.731}
